In [150]:
%matplotlib inline

from __future__ import division, print_function

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.special import factorial
from scipy.special import gamma
import time

#Typical plot parameters that make for pretty plots
mpl.rcParams['figure.figsize'] = (8,8)
mpl.rcParams['font.size'] = 15.0
mpl.rc('text', usetex='true') 

In [151]:
np.random.seed(int((time.time()%1)*1.0e8))

### Monte Carlo Integration

---

Useful to integrate when your dimension d > 10 or so.

If you sample function with N points, the error scales as error (variance) $\sim 1/\sqrt{N}$.  Slow, but guarenteed.  Make sure to vary point number, like use N and 2N when doing calculations for a good error comparison.  Also, you should run multiple independent simulations (different RNG seeds, of course!) to get a better answer, more robust error.  To do this, run multiple copies on different cores (easy to parallelize!), can plot distribution of answers to get answer, error from width of distribution.  We expect this error to decrease as the error equation given above.

Pros:
```
1. Good for large number of dimensions
2. error ~ 1/sqrt(N)
3. easy to run multiple independent simulations for comparison/error estimate
4. easy to parallelize
5. easy to check point (save state if calculation is interupted)
    -Ex: every x steps, print out RNG seed, point information to some file
```

Cons:
```
1. Can be slow for large N
2. Can require large N to beat down errors
```

Ex. Estimating pi using Monte Carlo Integration
```
1. Draw random x, y from [0,1] uniformly
2. Check to see if it's within the circle (if x^2 + y^2 < 1)
3. Count total points, total points within circle
4: (area of circle quadrant)/(area of 1 by 1 square) == (number of points within circle)/(total number of points)
```

Algorithms like this are really easy to generalize to a higher dimension.  Imagine some 10-sphere of "radius" 1.  Finding it's 10-volume (whatever that is) via Monte Carlo integration uses the same scheme as above but the if statement becomes
```
if x1^2 + x2^2 + ... + x10^2 < 1:
    area_count++
```

In [152]:
def volume4Dsphere(radius, N):
    """
    Perform a monte carlo integration to compute the radius of a 4d "sphere"
    using N samples
    """
    
    # Sanity checks
    if radius <= 0.0:
        print("ERROR: Radius must be greater than 0.")
        return -1
    if N <= 0:
        print("ERROR: Number of integration points must be greater than 0")
        return -1
    if N > 1.0e6:
        print("My computer can't handle this!")
        return -1
    
    # Draw 4 random deviates from 0 to radius
    # These sample 1/16 of sphere volume... like the equivalent of the 1st quadrant of the sphere
    # Volume of "1st quadrant" goes as 16*radius^4 (like )
    x = np.random.uniform(low=0.0, high=radius, size=N)
    y = np.random.uniform(low=0.0, high=radius, size=N)
    z = np.random.uniform(low=0.0, high=radius, size=N)
    w = np.random.uniform(low=0.0, high=radius, size=N)

    return 16.0*np.power(radius,4.0)*np.sum(x**2 + y**2 + z**2 + w**2 <= radius**2.0)/N

In [153]:
def trueVolume(radius):
    """
    Returns the true volume of a 4d sphere using the formula from
    https://en.wikipedia.org/wiki/N-sphere
    """
    return np.power(np.pi,2.0)*np.power(radius,4.)/gamma(3.0)

In [154]:
radius = 30.0
volume4Dsphere(radius,10000)

4000752.0

In [155]:
trueVolume(radius)

3997189.78244119

# Metropolis-Hastings Algorithm
### The meat and potatoes of MCMC

https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm

or 

http://www.life.illinois.edu/dietze/Lectures2012/Lesson12_Metropolis.pdf

Problem: Transformation method only works in 1D and the rejection method can be very inefficient.

Solution:  Sample around some x0 where you try to find the regions of maximal probability to make good usage of finite computer time.

---

Authors were simulating hard sphere gas physics.  Had a box with periodic boundary conditions and were seeing how the gas particles moved.  Use some maximum step size and drew random numbers to update positions to sample important configuration since we don't care about unlikely states.

### Algorithm

---

Always go downhill, sometimes go uphill (relative to -log probability minima) to prevent you from always getting stuck in local minima

```
For the case of gas particles with some interaction between them that gives the energy

1) We calculate the change in energy caused by the trial move

2) If Delta_E < 0: # Always go downhill
       accept move, put particle in new position
       X = X_new
       Y = Y_new
   else: # delta_E >= 0... sometimes accept uphill move
       get a random number U3 between 0 and 1
       if(U3 < exp(-Delta_E/kT)): # exp(-Delta_E/kT) is 0 <= P(Delta_E) <= 1: acceptance probability
           # accept the move
           X = X_new
           Y = Y_new
       else:
           pass # do nothing X=X, Y=Y, configuration is the same
           
we compute new position via X_old + alpha*U1

where alpha is the step size and U1 is a random deviate from [0,1]
```

Need to choose step size to make sure you sample the distribution.

Need to vary...
```
1) system size
2) Monte Carlo step size (x)
3) Number of steps
4) Starting configuration
5) Initial discarded steps
    - Do some initial steps, throw them out so your starting condition is reasonable.
    - This is the "burn in" or the "equilibrating the system"
        - If we started with bad initial condition, burn in can come to reasonable value
          so it doesn't screw up the computation too much
6) Vary how high (y) chain can jump... like acceptance ratio (Temperature T in above example)
```
to ensure your simulation is robust.  If you get similar results after varying everything, the result is likely solid.

### Averaging

---

$$
<F> = \frac{1}{N} \sum_{y=1}^N F_j
$$

using different simulation results.

### Autocorrelation

---

At some step t, look at a quantity c and then later, average it

$$
<c(t)c(t + \tau)>
$$
if it's small, its better.


From http://stats.stackexchange.com/questions/110268/why-is-it-desirable-to-have-low-auto-correlation-in-mcmc:
```
Autocorrelation is a measure of how much the value of a signal correlates to other values of that signal at different points in time. In the context of MCMC, autocorrelation is a measure of how independent different samples from your posterior distribution are – lower autocorrelation indicating more independent results.

When you have high autocorrelation the samples you've drawn don't accurately represent the posterior distribution and therefore don't provide as meaningful information for the solution to the problem. In other words, lower autocorrelation means higher efficiency in your chains and better estimates. A general rule would be that the lower your autocorrelation, the less samples you need for the method to be effective (but that might be oversimplifying).
```

But warning, you could have a low autocorrelation and be stuck in a local minima, so vary all the things mentioned above!

### Error estimates

---

Like other MCMCs, can use independent simulations to calculate variance for error bars

### System size scaling

---

Examine how parameters vary with system size.  For example, specific heat function becomes more peaked with system size.

## Example: Traveling Salesman Problem

---

Situation: Suppose you have some map of the US with a bunch of cities (nodes) and roads (edges) connecting them.  

Problem: How do we visit each city once while minimizing the length travelled?

This isn't as much as a problem for actual salemen as it is from things like shipping, airline travel, or electicity moving across a circuit.

You cannot solve this problem by generating all possible combinations.  For continental state capitols when you start in one, the number of combinations is 47!/2 which is a huge amount for small number of capitols.

### Simulated Annealing

---

First, label cities for simplicity
```
Inits:
1) Start with some random tour with length L
2) Pick some "temperature" larger than L
    We have some factor exp(-Delta_L/T) that governs how often we can go uphill
Loop:
3) Get the new step (trial configuration)
    - For example, select 2 random cities, flip sequence
4) Use Metropolis Algorithm 
    - If L goes less, always accept
    - If it increases, accept it with P = exp(-Delta_L/T)
    - Back to 3) to get new trial step
5) After some N Metropolis Algorithm step, set L = L - Delta_L (Reduce temp step)

To clarify, there are two loops:  Inner Metropolis loop then outer temperature loop
```